In [1]:
!pip install nltk

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [4]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""

In [48]:
document = document.lower()

In [5]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
tokens = word_tokenize(document.lower())

In [8]:
vocab = {'<UNK>' : 0}

In [10]:
for token in set(tokens):
  if token not in vocab:
    vocab[token] = len(vocab)

In [12]:
len(vocab)

289

In [34]:
input_sentences = document.split('\n')

In [16]:
def text_to_indices(sentence, vocab = vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<UNK>'])
  return numerical_sentence

In [49]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower())))

In [50]:
len(input_numerical_sentences[0]), input_numerical_sentences[0]

(3, [76, 287, 91])

In [51]:
len(input_numerical_sentences)

78

In [53]:
training_sequence = []

for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[: i+1])

In [54]:
training_sequence[:5]

[[76, 287], [76, 287, 91], [183, 8], [183, 8, 287], [183, 8, 287, 244]]

In [55]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

In [56]:
mx_len = max(len_list)

In [59]:
padded_traning_sequence = []

for sequence in training_sequence:
  padded_traning_sequence.append([0]*(mx_len - len(sequence)) + sequence)

In [62]:
len(padded_traning_sequence[0])

62

In [66]:
padded_traning_sequence = torch.tensor(padded_traning_sequence)

In [67]:
padded_traning_sequence

tensor([[  0,   0,   0,  ...,   0,  76, 287],
        [  0,   0,   0,  ...,  76, 287,  91],
        [  0,   0,   0,  ...,   0, 183,   8],
        ...,
        [  0,   0,   0,  ..., 144, 200, 266],
        [  0,   0,   0,  ..., 200, 266,   1],
        [  0,   0,   0,  ..., 266,   1,  30]])

In [68]:
x = padded_traning_sequence[:, :-1]
y = padded_traning_sequence[:, -1]

In [74]:
class CustomDataset(Dataset):

  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [75]:
dataset = CustomDataset(x, y)

In [78]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

In [113]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

### understanding how the shape is changing (with one sentence)

In [156]:
p = nn.Embedding(len(vocab), 100)

emb = p(dataset[0][0]) # shape is for one sen -> 61,100

q = nn.LSTM(100, 150, batch_first=True)

lmso = q(emb)[1][0] # shape is 1,150 for one sen

r = nn.Linear(150, len(vocab))

r(lmso).shape # shape is 1,289

torch.Size([1, 289])

#### understanding how the shape is chaing (with batch of 32)

In [163]:
p = nn.Embedding(len(vocab), 100)

emb = p(batch_x) # shape is for one sen -> 32,61,100

q = nn.LSTM(100, 150, batch_first=True)

lmso = q(emb)[1][0] # 1,32,150

# because there is one extra dim, we need to squeeze it (reduce one dim)
lmso = lmso.squeeze(0)

r = nn.Linear(150, len(vocab))

r(lmso).shape

torch.Size([32, 289])

In [114]:
model = LSTMModel(len(vocab))

In [115]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [116]:
model = model.to(device)

epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [134]:
for batch_x, _ in dataloader:
  print(batch_x.shape)
  break

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


torch.Size([32, 61])


In [118]:
for epoch in range(epochs):
  epoch_loss = []

  for batch_x, batch_y in dataloader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    output = model(batch_x)

    loss = criterion(output, batch_y)

    with torch.no_grad():
      epoch_loss.append(loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f'loss at epoch {epoch+1} is {np.mean(epoch_loss)}')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss at epoch 1 is 4.890951681137085
loss at epoch 2 is 4.518069394429525
loss at epoch 3 is 4.119919872283935
loss at epoch 4 is 3.712356193860372
loss at epoch 5 is 3.347192923227946
loss at epoch 6 is 2.9929926713307697
loss at epoch 7 is 2.6758057196935017
loss at epoch 8 is 2.3678054014841714
loss at epoch 9 is 2.097825264930725
loss at epoch 10 is 1.8460829933484395
loss at epoch 11 is 1.625195332368215
loss at epoch 12 is 1.4301431934038797
loss at epoch 13 is 1.255353832244873
loss at epoch 14 is 1.0960170725981395
loss at epoch 15 is 0.9780794044335683
loss at epoch 16 is 0.8512605408827464
loss at epoch 17 is 0.7574940979480743
loss at epoch 18 is 0.6783250431219737
loss at epoch 19 is 0.6013996183872223
loss at epoch 20 is 0.5451655904452006
loss at epoch 21 is 0.4890124350786209
loss at epoch 22 is 0.4474372853835424
loss at epoch 23 is 0.40991798440615335
loss at epoch 24 is 0.3831833302974701
loss at epoch 25 is 0.34816407710313796
loss at epoch 26 is 0.32522489031155905


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss at epoch 30 is 0.2533450131615003
loss at epoch 31 is 0.23996060515443485
loss at epoch 32 is 0.23364357501268387
loss at epoch 33 is 0.22309893096486727


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss at epoch 34 is 0.21594822704792022
loss at epoch 35 is 0.20829227864742278
loss at epoch 36 is 0.1963872934381167
loss at epoch 37 is 0.19397781069080036
loss at epoch 38 is 0.18343321482340494
loss at epoch 39 is 0.18325840607285498
loss at epoch 40 is 0.17941149498025577
loss at epoch 41 is 0.17236628234386445
loss at epoch 42 is 0.1667420081794262
loss at epoch 43 is 0.16311525069177152
loss at epoch 44 is 0.16289928257465364
loss at epoch 45 is 0.161185231183966
loss at epoch 46 is 0.15339320041239263
loss at epoch 47 is 0.1518453133602937
loss at epoch 48 is 0.15108390003442765
loss at epoch 49 is 0.14796588098009428
loss at epoch 50 is 0.14961615179975826


In [119]:
def prediction(model, text, vocab = vocab):

  tokenized_text = word_tokenize(text)

  numerical_text = text_to_indices(tokenized_text)

  padded_text = torch.tensor([0]*(mx_len-len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  output = model(padded_text)

  idx = output.argmax(dim=1)

  return text + " " + list(vocab.keys())[idx]

In [131]:
n_token = 10
input_text = 'What is the criteria'
for i in range(5):
  output_text = prediction(model, input_text)
  print(output_text)
  input_text = output_text

What is the criteria to
What is the criteria to get
What is the criteria to get the
What is the criteria to get the certificate
What is the criteria to get the certificate ?
